# Модель GARCH(1,1) Skewed-t

- Обучение на returns
- Прогноз волатильности (скользящее окно)
- Tail risk

Шаблон содержит структуру для быстрого запуска, визуализирует сходится ли модель.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Попытка импорта arch для GARCH
try:
    from arch import arch_model
    print("✅ Библиотеки загружены (arch доступен)")
    ARCH_AVAILABLE = True
except ImportError:
    print("⚠️ Библиотека 'arch' не установлена. Установите: pip install arch")
    ARCH_AVAILABLE = False


✅ Библиотеки загружены (arch доступен)


## Загрузка данных


In [3]:
import json

DATA_DIR = Path('../02_feature_engineering/data/features')

# Вариант 1: Получить список тикеров из файлов в директории
all_files = list(DATA_DIR.glob("*_with_targets.parquet"))
tickers = [f.stem.replace("_with_targets", "") for f in all_files]

# Вариант 2: Или загрузить из metadata (более надёжный способ)
# with open(Path('../config/tickers_metadata.json'), 'r') as f:
#     metadata = json.load(f)
# tickers = [k for k in metadata.keys() if not k.startswith('_')]

# Загрузка всех данных в словарь
all_data = {}
for ticker in tickers:
    file_path = DATA_DIR / f"{ticker}_with_targets.parquet"
    if file_path.exists():
        df = pd.read_parquet(file_path)
        # Подготовка данных для GARCH
        returns = df['log_return'].dropna() * 100  # Масштабируем для стабильности
        all_data[ticker] = {
            'df': df,
            'returns': returns
        }
        print(f"✅ {ticker}: {len(returns)} наблюдений, mean={returns.mean():.4f}, std={returns.std():.4f}")

print(f"\n📊 Загружено {len(all_data)} тикеров: {list(all_data.keys())}")


✅ AFKS: 1301 наблюдений, mean=-0.0517, std=2.5570
✅ AFLT: 1301 наблюдений, mean=-0.0116, std=2.5475
✅ ALRS: 1301 наблюдений, mean=-0.0510, std=2.1944
✅ BELU: 1289 наблюдений, mean=-0.0918, std=6.3728
✅ BSPB: 1295 наблюдений, mean=0.1606, std=2.7076
✅ CHMF: 1301 наблюдений, mean=-0.0083, std=2.3466
✅ FIVE: 242 наблюдений, mean=-0.0585, std=1.8532
✅ GAZP: 1301 наблюдений, mean=-0.0293, std=2.6512
✅ GMKN: 1297 наблюдений, mean=-0.0337, std=1.9813
✅ HYDR: 1301 наблюдений, mean=-0.0500, std=1.8148
✅ IMOEX: 1253 наблюдений, mean=-0.0076, std=1.9141
✅ IRAO: 1299 наблюдений, mean=-0.0502, std=2.3502
✅ LENT: 990 наблюдений, mean=0.0461, std=2.5738
✅ LKOH: 1301 наблюдений, mean=0.0228, std=1.9893
✅ MAGN: 1301 наблюдений, mean=-0.0244, std=2.2061
✅ MGNT: 1301 наблюдений, mean=-0.0371, std=2.2130
✅ MTSS: 1300 наблюдений, mean=-0.0382, std=2.0580
✅ NLMK: 1301 наблюдений, mean=-0.0413, std=2.2532
✅ NVTK: 1297 наблюдений, mean=-0.0051, std=2.4377
✅ OZON: 1205 наблюдений, mean=0.0257, std=3.3049
✅ PIK

## Обучение GARCH(1,1) модели


In [4]:
if ARCH_AVAILABLE:
    garch_models = {}
    
    for ticker, data in all_data.items():
        returns = data['returns']
        
        # Создаем и обучаем GARCH(1,1) модель с Skewed-t распределением
        model = arch_model(returns, vol='GARCH', p=1, q=1, dist='skewt', rescale=False)
        
        try:
            fitted_model = model.fit(disp='off', show_warning=False)
            
            # Извлечение условной волатильности
            conditional_vol = fitted_model.conditional_volatility
            
            # Прогноз волатильности на 1 шаг вперед
            forecast = fitted_model.forecast(horizon=1)
            forecast_variance = forecast.variance.iloc[-1, 0]
            forecast_vol = np.sqrt(forecast_variance)
            
            garch_models[ticker] = {
                'model': fitted_model,
                'conditional_vol': conditional_vol,
                'forecast_vol': forecast_vol
            }
            print(f"✅ {ticker}: прогноз волатильности = {forecast_vol:.4f}%")
            
        except Exception as e:
            print(f"❌ {ticker}: ошибка обучения - {e}")
    
    print(f"\n📊 Успешно обучено моделей: {len(garch_models)}/{len(all_data)}")
    
else:
    print("❌ GARCH модель недоступна. Установите библиотеку 'arch'.")

✅ AFKS: прогноз волатильности = 2.5794%
✅ AFLT: прогноз волатильности = 2.5955%
✅ ALRS: прогноз волатильности = 2.1768%
✅ BELU: прогноз волатильности = 1.6288%
✅ BSPB: прогноз волатильности = 2.2044%
✅ CHMF: прогноз волатильности = 2.2104%
✅ FIVE: прогноз волатильности = 2.6901%
✅ GAZP: прогноз волатильности = 2.1962%
✅ GMKN: прогноз волатильности = 2.6193%
✅ HYDR: прогноз волатильности = 1.8991%
✅ IMOEX: прогноз волатильности = 1.9695%
✅ IRAO: прогноз волатильности = 2.2273%
✅ LENT: прогноз волатильности = 1.8268%
✅ LKOH: прогноз волатильности = 1.8066%
✅ MAGN: прогноз волатильности = 2.2918%
✅ MGNT: прогноз волатильности = 1.9350%
✅ MTSS: прогноз волатильности = 1.5662%
✅ NLMK: прогноз волатильности = 2.3833%
✅ NVTK: прогноз волатильности = 2.2520%
✅ OZON: прогноз волатильности = 1.7603%
✅ PIKK: прогноз волатильности = 5.8839%
✅ PLZL: прогноз волатильности = 1.5502%
✅ ROSN: прогноз волатильности = 1.9977%
✅ RTKM: прогноз волатильности = 2.4140%
✅ SBER: прогноз волатильности = 2.0615%

## Walk-forward прогнозирование


In [5]:
if ARCH_AVAILABLE:
    all_forecasts = {}
    
    for ticker, data in all_data.items():
        returns = data['returns']
        df = data['df']
        
        # Walk-forward прогнозирование на последних 100 днях
        train_size = len(returns) - 100
        if train_size < 100:  # Недостаточно данных
            print(f"⚠️ {ticker}: недостаточно данных для walk-forward")
            continue
            
        forecasts = []
        
        print(f"🔄 {ticker}: Walk-forward прогнозирование...")
        
        for i in range(train_size, len(returns)):
            train_data = returns.iloc[:i]
            
            try:
                model_temp = arch_model(train_data, vol='Garch', p=1, q=1, dist='skewt', rescale=False)
                res_temp = model_temp.fit(disp='off', show_warning=False, update_freq=0)
                
                forecast_temp = res_temp.forecast(horizon=1)
                forecasts.append(np.sqrt(forecast_temp.variance.iloc[-1, 0]))
            except:
                forecasts.append(np.nan)
        
        # Добавляем прогнозы в DataFrame
        forecast_dates = df['date'].iloc[-100:]
        forecast_df = pd.DataFrame({
            'date': forecast_dates.values,
            'garch_forecast': forecasts
        })
        
        all_forecasts[ticker] = forecast_df
        print(f"✅ {ticker}: {len(forecasts)} прогнозов")
    
    print(f"\n📊 Walk-forward завершен для {len(all_forecasts)} тикеров")
    
else:
    print("❌ Walk-forward недоступен без библиотеки 'arch'.")


🔄 AFKS: Walk-forward прогнозирование...
✅ AFKS: 100 прогнозов
🔄 AFLT: Walk-forward прогнозирование...
✅ AFLT: 100 прогнозов
🔄 ALRS: Walk-forward прогнозирование...
✅ ALRS: 100 прогнозов
🔄 BELU: Walk-forward прогнозирование...
✅ BELU: 100 прогнозов
🔄 BSPB: Walk-forward прогнозирование...
✅ BSPB: 100 прогнозов
🔄 CHMF: Walk-forward прогнозирование...
✅ CHMF: 100 прогнозов
🔄 FIVE: Walk-forward прогнозирование...
✅ FIVE: 100 прогнозов
🔄 GAZP: Walk-forward прогнозирование...
✅ GAZP: 100 прогнозов
🔄 GMKN: Walk-forward прогнозирование...
✅ GMKN: 100 прогнозов
🔄 HYDR: Walk-forward прогнозирование...
✅ HYDR: 100 прогнозов
🔄 IMOEX: Walk-forward прогнозирование...
✅ IMOEX: 100 прогнозов
🔄 IRAO: Walk-forward прогнозирование...
✅ IRAO: 100 прогнозов
🔄 LENT: Walk-forward прогнозирование...
✅ LENT: 100 прогнозов
🔄 LKOH: Walk-forward прогнозирование...
✅ LKOH: 100 прогнозов
🔄 MAGN: Walk-forward прогнозирование...
✅ MAGN: 100 прогнозов
🔄 MGNT: Walk-forward прогнозирование...
✅ MGNT: 100 прогнозов
🔄 MTSS

## Сохранение результатов


In [6]:
if ARCH_AVAILABLE:
    OUTPUT_DIR = Path('data') / 'models'
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    saved_count = 0
    for ticker, forecast_df in all_forecasts.items():
        output_path = OUTPUT_DIR / f"{ticker}_garch_forecasts.parquet"
        forecast_df.to_parquet(output_path, index=False)
        saved_count += 1
    
    print(f"✅ Сохранено {saved_count} файлов в {OUTPUT_DIR}")
else:
    print("⚠️ Нечего сохранять без GARCH модели")


✅ Сохранено 31 файлов в data\models
